Needed imports and utility

In [21]:
import os
import ansys.speos.script as script
import ansys.speos.core as core

tests_data_path = os.path.join(os.path.join(os.path.abspath(''), os.path.pardir), "assets")

def clean_all_dbs(speos_client: core.SpeosClient):
    for item in (
        speos_client.jobs().list()
        + speos_client.scenes().list()
        + speos_client.simulation_templates().list()
        + speos_client.sensor_templates().list()
        + speos_client.source_templates().list()
        + speos_client.intensity_templates().list()
        + speos_client.spectrums().list()
        + speos_client.vop_templates().list()
        + speos_client.sop_templates().list()
        + speos_client.parts().list()
        + speos_client.bodies().list()
        + speos_client.faces().list()
    ):
        item.delete()

tests_data_path = os.path.join(os.path.join(os.path.abspath(''), os.path.pardir), "assets")

Create connection with speos rpc server

In [22]:
speos = core.Speos(host="localhost", port=50051)
clean_all_dbs(speos.client)

New project

In [23]:
p = script.Project(speos=speos)
print(p)

ansys.api.speos.scene.v2.Scene
{
    "name": "",
    "description": "",
    "metadata": {},
    "part_guid": "",
    "sources": [],
    "sensors": [],
    "simulations": [],
    "materials": [],
    "scenes": []
}


# Create
Create locally  
The mention "local: " is added when printing the sensor  
For sensors, it is always important to set properties.

In [24]:
distortion_file_path = os.path.join(tests_data_path, "CameraInputFiles", "CameraDistortion_130deg.OPTDistortion")
transmittance_file_path = os.path.join(tests_data_path, "CameraInputFiles", "CameraTransmittance.spectrum")
blue_spectrum_path = os.path.join(tests_data_path, "CameraInputFiles", "CameraSensitivityBlue.spectrum")
green_spectrum_path = os.path.join(tests_data_path, "CameraInputFiles", "CameraSensitivityGreen.spectrum")
red_spectrum_path = os.path.join(tests_data_path, "CameraInputFiles", "CameraSensitivityRed.spectrum")

sensor1 = p.create_sensor(name="Sensor.1")
sensor1.set_camera().set_distortion_file_uri(uri=distortion_file_path)\
                    .set_mode_photometric().set_transmittance_file_uri(uri=transmittance_file_path)\
                                           .set_mode_color().set_blue_spectrum_file_uri(uri=blue_spectrum_path)\
                                                            .set_green_spectrum_file_uri(uri=green_spectrum_path)\
                                                            .set_red_spectrum_file_uri(uri=red_spectrum_path)
sensor1.set_camera_properties()
print(sensor1)

local: ansys.api.speos.scene.v2.Scene.SensorInstance
{
    "name": "Sensor.1",
    "camera_properties": {
        "axis_system": [
            0.0,
            0.0,
            0.0,
            1.0,
            0.0,
            0.0,
            0.0,
            1.0,
            0.0,
            0.0,
            0.0,
            1.0
        ],
        "layer_type_none": {},
        "trajectory_file_uri": ""
    },
    "description": "",
    "metadata": {},
    "sensor_guid": "",
    "result_file_name": ""
}
local: ansys.api.speos.sensor.v1.SensorTemplate
{
    "camera_sensor_template": {
        "sensor_mode_photometric": {
            "acquisition_integration": 0.01,
            "transmittance_file_uri": "c:\\ANSYSDev\\NoBackup\\pyspeos\\tests\\jupyter_notebooks\\..\\assets\\CameraInputFiles\\CameraTransmittance.spectrum",
            "gamma_correction": 2.2,
            "png_bits": "PNG_16",
            "color_mode_color": {
                "red_spectrum_file_uri": "c:\\ANSYSDev\\NoBa

Push it to the server.  
Now that it is committed to the server, the mention "local: " is no more present when printing the sensor.  

In [ ]:
sensor1.commit()
print(sensor1)

Another example by setting more characteristics

In [ ]:
distortion_file_path = os.path.join(tests_data_path, "CameraInputFiles", "CameraDistortion_130deg.OPTDistortion")
blue_spectrum_path = os.path.join(tests_data_path, "CameraInputFiles", "CameraSensitivityBlue.spectrum")
green_spectrum_path = os.path.join(tests_data_path, "CameraInputFiles", "CameraSensitivityGreen.spectrum")
red_spectrum_path = os.path.join(tests_data_path, "CameraInputFiles", "CameraSensitivityRed.spectrum")

sensor2 = p.create_sensor(name="Sensor.2")
sensor2.set_camera().set_distortion_file_uri(uri=distortion_file_path)\
                    .set_mode_photometric().set_mode_color().set_blue_spectrum_file_uri(uri=blue_spectrum_path)\
                                                            .set_green_spectrum_file_uri(uri=green_spectrum_path)\
                                                            .set_red_spectrum_file_uri(uri=red_spectrum_path)
sensor2.set_camera().set_focal_length(5.5).set_height(value=6).set_width(value=6)
sensor2.set_camera_properties().set_axis_system([20,10,40,1,0,0,0,1,0,0,0,1]).set_layer_type_source()
sensor2.commit()
print(sensor2)

## Default values
Some default values are available when applicable in every methods and class.

# Read
## Source Instance
A mention "local: " is added if it is not yet committed to the server

In [ ]:
print(sensor1)

## Project

In [ ]:
print(p)

# Update
Tipp: if you are manipulating a source already committed, don't forget to commit your changes.  
If you don't, you will still only watch what is committed on the server.

In [ ]:
sensor1.set_camera().set_f_number(value=11)
sensor1.set_camera_properties().set_axis_system([17, 10, 10, 1, 0, 0, 0, 1, 0, 0, 0, 1])
sensor1.commit()
print(sensor1)

Be careful to have something coherent when updating.  
Example just after source of type luminaire but with rayfile properties.

In [ ]:
#TODO source1.set_rayfile_properties()

## Reset
Possibility to reset local values from the one available in the server.

In [ ]:
sensor1.set_camera().set_mode_geometric() # set rayfile_properties but no commit
sensor1.reset() # reset -> this will apply the server value to the local value
sensor1.delete() # delete (to display the local value with the below print)
print(sensor1._sensor_template)

# Delete
Once the data is deleted from the server, you can still work with local data and maybe commit later.

In [ ]:
sensor2.delete()
print(sensor2)

In [ ]:
sensor1.delete()

# More content
## Irradiance sensor

In [ ]:
#TODO
#ray_file_path = os.path.join(tests_data_path, "Rays.ray")
#
#source3 = p.create_source(name="Source.3")
#source3.set_rayfile().set_ray_file_uri(uri=ray_file_path)
#source3.set_rayfile_properties()
#source3.commit()
#print(source3)

In [ ]:
#source3.set_rayfile().set_flux_luminous()
#source3.commit()
#print(source3)

In [ ]:
#source3.delete()

## radiance sensor

In [ ]:
#TODO
#source4 = p.create_source(name="Source.4")
#source4.set_surface().set_exitance_constant()
#source4.set_surface_properties().set_exitance_constant_properties([(script.GeoRef.from_native_link("TheBodyB/TheFaceF"), False), 
#                                                                   (script.GeoRef.from_native_link("TheBodyB/TheFaceG"), True)])
#source4.commit()
#print(source4)

In [ ]:
#source4.set_surface().set_flux_luminous_intensity()
#source4.set_surface().set_intensity().set_gaussian().set_gaussian_properties([10,50,20,1,0,0,0,1,0,0,0,1])
#source4.commit()
#print(source4)

In [ ]:
#source4.delete()
#print(source4)

# Cleanup

In [ ]:
clean_all_dbs(speos.client)